# Imports

In [0]:
import os
import sys
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
from google.colab import drive

In [0]:
import warnings
warnings.filterwarnings('ignore')

# Data Gathering

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_path = 'drive/My Drive/dna_data/Download'

In [0]:
def process(lines=None):
  ks = ['id', 'sequence', 'optional', 'quality']
  return {k: v for k, v in zip(ks, lines)}

In [0]:
def read_fq(file_path):
  fastq_list = []
  n = 4
  with open(file_path, 'r') as fh:
    lines = []
    for line in fh:
      lines.append(line.rstrip())
      if len(lines) == n:
        record = process(lines)
        fastq_list.append(record)
        lines = []
        
  return fastq_list

In [0]:
def dir_to_df(dir_path):
  directory = os.fsencode(dir_path)
  files_list = []

  for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".fq"):   
      files_list.append(read_fq(dir_path + '/' + filename)) 
    else:
      continue

  df = pd.DataFrame({'id' : [], 'sequence': [], 'optional': [], 'quality': []})

  for element in files_list:
    temp_df = pd.DataFrame(element)
    df = df.append(temp_df, ignore_index = True)

  return df

In [0]:
telomeric = dir_to_df(data_path + '/Labeled/telomeric')

In [123]:
telomeric.shape

(180000, 4)

In [0]:
non_telomeric = dir_to_df(data_path + '/Labeled/non-telomeric')

In [125]:
non_telomeric.shape

(405545, 4)

In [0]:
telomeric.drop(columns=['optional', 'quality'], inplace=True)
telomeric['is_telomeric'] = 1
non_telomeric.drop(columns=['optional', 'quality'], inplace=True)
non_telomeric['is_telomeric'] = 0

In [0]:
# undersampling with the size of telomeric dataset
# non_telomeric.sample(frac=1).reset_index(drop=True)
# non_telomeric = non_telomeric[:telomeric.shape[0]]
# print('non_telomeric :', non_telomeric.shape)

In [0]:
both_data = telomeric.append(non_telomeric, ignore_index = True)

In [169]:
both_data.shape

(585545, 3)

In [0]:
# shuffle dataset
both_data = both_data.sample(frac=1).reset_index(drop=True)

In [0]:
# both_data.head(20)

# Data Preprocessing

##K-mers method

In [0]:
# k-mers counting method
def getKmers(sequence, size=6):
  return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [0]:
both_data['words'] = both_data.apply(lambda x: getKmers(x['sequence']), axis=1)
both_data = both_data.drop('sequence', axis=1)

In [0]:
gene_texts = list(both_data['words'])
for item in range(len(gene_texts)):
  gene_texts[item] = ' '.join(gene_texts[item])

y = both_data.iloc[:, 1].values  

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4, 4))
X = cv.fit_transform(gene_texts)

##Vectorized

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
def tokenize_pad(df, vocab_size, max_len):
  vocab = vocab_size
  tokenizer = Tokenizer(num_words=vocab, char_level=True, oov_token='dummy')
  tokenizer.fit_on_texts(df)
  tokenized_seqs = tokenizer.texts_to_sequences(df)
  padded_seqs = pad_sequences(tokenized_seqs, padding='post', maxlen=max_len)

  return padded_seqs

In [0]:
X_train = tokenize_pad(both_data['sequence'], 5, 250)

In [174]:
X_train.shape

(585545, 250)

In [0]:
y_train = both_data['is_telomeric']

Test Data Preprocessing

In [0]:
test_class_df = pd.DataFrame(read_fq(data_path + '/Test/test_tel_no-tel.fq'))

In [0]:
test_class = tokenize_pad(test_class_df['sequence'], 5, 250)

In [36]:
test_class.shape

(146890, 250)

In [0]:
test_mixed_df = pd.DataFrame(read_fq(data_path + '/Labeled/mixed/file_227_hs10_100bp_10x_chr1@tel1-3087_tel2-3437.fq'))
test_mixed_test_df = pd.DataFrame(read_fq(data_path + '/Test/test_mixed/file_001_hs10_100bp_10x_chr1.fq'))

In [0]:
test_mixed = tokenize_pad(test_mixed_df['sequence'], 5, 250)
test_mixed_test = tokenize_pad(test_mixed_test_df['sequence'], 5, 250)

In [149]:
print(test_mixed.shape, test_mixed_test.shape)

(23600, 250) (23620, 250)


In [0]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(padded_seqs, 
#                                                     y, 
#                                                     test_size = 0.20, 
#                                                     random_state=42)

In [0]:
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Models

##Naive Bayes

In [0]:
 X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state=42)

In [0]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [0]:
y_pred = classifier.predict(X_test)

In [0]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted      0      1
Actual                 
0          35860      0
1              0  36140
accuracy = 1.000 
precision = 1.000 
recall = 1.000 
f1 = 1.000


##RNN

In [0]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import Masking, Embedding

In [0]:
model = Sequential()

model.add(Masking(mask_value=0., input_shape=(250, 1)))

model.add(Conv1D(filters=8, kernel_size=4, activation='relu'))

model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=8, kernel_size=8, activation='relu'))

model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Conv1D(filters=8, kernel_size=8, activation='relu'))

model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

# model.add(Masking(mask_value=0., input_shape=(250,)))

# model.add(Embedding(vocab, 8, input_length=None)) 

# model.add(BatchNormalization())
# model.add(GRU(128, return_sequences = True))
# model.add(GRU(128, return_sequences = True))
# model.add(GRU(16, return_sequences = True))
# model.add(GRU(16))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(8, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
# use weighted classes when data is not undersampled
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [321]:
model.fit(X_train[:,:,np.newaxis], y_train,
          batch_size = 128, epochs = 10, class_weight=class_weights,
          verbose = 1)

Train on 585545 samples
Epoch 1/10
585545/585545 [==============================] - 43s 73us/sample - loss: 0.1066 - acc: 0.9589
Epoch 2/10
 84352/585545 [===>..........................] - ETA: 36s - loss: 0.0376 - acc: 0.9888

KeyboardInterrupt: ignored

# Predictions

In [0]:
def classify_binary(pred):
  pred[pred >= 0.5] = 1
  pred[pred < 0.5] = 0

  return pred

In [0]:
def calculate_tel_length(tel_num, bp = 100, coverage = 10):
  tel_length = ((tel_num / coverage) * bp) / 2
  return tel_length

Classification of the test set

In [0]:
pred_class = model.predict(test_class[:,:,np.newaxis])

In [272]:
np.sum(classify_binary(pred_class))

45263.0

In [0]:
pred_class = classify_binary(pred_class)
test_class_df['prediction'] = pred_class

Predictions for the mixed set

In [0]:
pred_mixed = model.predict(test_mixed[:,:,np.newaxis])

In [323]:
pred_mixed_len = np.sum(classify_binary(pred_mixed))
pred_mixed_len

637.0

In [324]:
calculate_tel_length(pred_mixed_len)
# 3262

3185.0

In [325]:
pred_mixed_test = model.predict(test_mixed_test[:,:,np.newaxis])

calculate_tel_length(np.sum(classify_binary(pred_mixed_test)))

170.0

In [0]:
def predict_mixed(dir_path):
  directory = os.fsencode(dir_path)
  
  length_dict = {}

  for file in tqdm(os.listdir(directory)):
    filename = os.fsdecode(file)
    
    if filename.endswith(".fq"):
      temp_df = pd.DataFrame(read_fq(dir_path + '/' + filename))
      temp = tokenize_pad(temp_df['sequence'], 5, 250)
      pred = model.predict(temp[:,:,np.newaxis])
      pred = classify_binary(pred)
      length = calculate_tel_length(np.sum(pred))
      length_dict[filename] = length
    else:
      continue
    
  return length_dict

In [0]:
cnn_pred_lengths = predict_mixed(data_path + '/Test/test_mixed')

In [250]:
cnn_pred_lengths.popitem()

('file_143_hs10_100bp_10x_chr2.fq', 25.0)